# Rapid API post

- title: Rapid API post
- toc: true 
- badges: true
- comments: true
- categories: [jupyter]

In [2]:
import java.net.URI;
import java.net.http.HttpClient;
import java.net.http.HttpRequest;
import java.net.http.HttpResponse;

//RapidAPI header  https://rapidapi.com/Coinranking/api/coinranking1/
HttpRequest request = HttpRequest.newBuilder()
    .uri(URI.create("https://coinranking1.p.rapidapi.com/api"))
    .header("x-rapidapi-key", "3bef89a477msh7eecb5e7498cd51p1095f2jsn9b36f511d7ec")
    .header("x-rapidapi-host", "coinranking1.p.rapidapi.com")
    .method("GET", HttpRequest.BodyPublishers.noBody())
    .build();

//RapidAPI request and response
HttpResponse<String> response = HttpClient.newHttpClient().send(request, HttpResponse.BodyHandlers.ofString());

//RapidAPI Body
System.out.println(response.body());

{"message":"Endpoint\/api does not exist"}
